In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Importando
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [3]:
import pandas as pd
df2 = pd.read_csv('/content/drive/MyDrive/Estudos /Machine Learning/exemplo3.csv')
df2.head()

,idade,conta_corrente,sexo,risco
0,21.257389,783.127911,masculino,ruim
1,21.766573,979.747521,feminino,bom
2,25.321033,1065.328054,feminino,bom
3,23.919268,1195.758078,feminino,bom
4,21.805298,1083.764450,feminino,bom


Nesse exemplo temos a coluna sexo, identificando o sexo de cada cliente como masculino ou feminino. As variáveis de input de um Classificador devem, necessariamente, ser o tipo númerico.

Para transformar as informações contidas na coluna sexo para o formato númerico, faremos a binarização da coluna através da classe **OneHotEncoder**.

Primeiro, faremos a seleção da variáveis de input do tipo object(str), ultlizando o comando DataFrame.select_dtypes(), passando para o parâmetro include a lista de tipos selecionados (nesse caso, o tipo object).

In [4]:
X = df2.drop('risco', axis=1)
y = df2.risco
#Selecionando somente a coluna que tem o d_types 'Object'
X.select_dtypes(exclude='object')

,idade,conta_corrente
0,21.257389,783.127911
1,21.766573,979.747521
2,25.321033,1065.328054
3,23.919268,1195.758078
4,21.805298,1083.764450
...,...,...
2495,63.840703,2847.204903
2496,57.367341,3238.149058
2497,65.785408,2603.620751
2498,52.944838,2985.948214


O resultado é um DataFrame contendo somente a coluna sexo.

Faremos a binarização desse DataFrame ultilizando OneHotEncoder presente no subpacote Sklearn.preprocessing.
Chamaremos de onehot o objeto da classe OneHotEncoderm passando os parâmetros sparse=False (para que os resultados não seja gerado no formato de uma matriz esparsa) e drop='first'(para eliminar redundância na binarização)

Aplicaremos o operador ultilizando o comando OneHptEncoder.fit_transform(), atribuindo a uma variável chamada X_bin.

In [5]:
from sklearn.preprocessing import OneHotEncoder

X_cat = X.select_dtypes(include = 'object')


onehot = OneHotEncoder(sparse_output=False, drop='first')
X_bin = onehot.fit_transform(X_cat)
X_num = X[['idade', 'conta_corrente']]

# Combinando as variáveis numéricas e as variáveis binárias
X_all = np.hstack((X_num, X_bin))
# Criando um DataFrame a partir do array combinado
df_final = pd.DataFrame(X_all, columns=['idade', 'conta_corrente', 'sexo_masculino'])
display(df_final)


,idade,conta_corrente,sexo_masculino
0,21.257389,783.127911,1.0
1,21.766573,979.747521,0.0
2,25.321033,1065.328054,0.0
3,23.919268,1195.758078,0.0
4,21.805298,1083.764450,0.0
...,...,...,...
2495,63.840703,2847.204903,0.0
2496,57.367341,3238.149058,1.0
2497,65.785408,2603.620751,1.0
2498,52.944838,2985.948214,1.0


In [6]:
#Fazendo a normalização dos numeros em 0 e 1
from sklearn.preprocessing import MinMaxScaler

X_num = X.select_dtypes(exclude = 'object')
minmax = MinMaxScaler()
X_num = minmax.fit_transform(X_num)

X_num


array([[0.05586473, 0.06720224],
       [0.06499436, 0.09938945],
       [0.12872564, 0.11339923],
       ...,
       [0.85424954, 0.36522222],
       [0.62401886, 0.42781034],
       [0.61473291, 0.54031447]])

Finalmente, uniremos os DataFrames X_bin e X_num em um só DataFrame chamado X_all. Para tanto, ultilizarmos o comando numpy.append() presente no pacote numpy

Faremos a impoertação do pacote a ele o apelido de np e aplicaremos a função append(), passando como parâmetro os array X_num e X_bin. O parâmetro axis=1 precisa ser acrescentado à função para explicitar que a junção derpa feita no eixo das colunas.

In [7]:
print(X_num.shape)
print(X_bin.shape)


(2500, 2)
(2500, 1)


In [8]:
#importando
#Juntando os dois arrays X_num e X_bin
import numpy as np
X_all = np.append(X_num, X_bin, axis=1)
X_all

array([[0.05586473, 0.06720224, 1.        ],
       [0.06499436, 0.09938945, 0.        ],
       [0.12872564, 0.11339923, 0.        ],
       ...,
       [0.85424954, 0.36522222, 1.        ],
       [0.62401886, 0.42781034, 1.        ],
       [0.61473291, 0.54031447, 1.        ]])

In [9]:
#Fazendo o treinemnto

#Treinando
X_train, X_test, y_train, y_test = train_test_split(X_all, y, test_size=1/3, random_state=42)

#Chamando o Classificador SVC
svc2 = SVC()
svc2.fit(X_train, y_train)

#Acuracia
acuracia = accuracy_score(y_test, svc2.predict(X_test))*100
print(f'{acuracia:.2f}%')

71.46%



Para ultilizarmos o classificar para prever a classe de um novo conjunto de dados, criaremos um DataFrame contendo cinco novos clientes com as seguintes informações.

In [10]:
df_new = pd.DataFrame({'idade': [20,25,50,35,75],
                       'conta_corrente':[800,4000,2200,3200,100],
                       'sexo': ['masculino','feminino','masculino','feminino','feminino']})
df_new


,idade,conta_corrente,sexo
0,20,800,masculino
1,25,4000,feminino
2,50,2200,masculino
3,35,3200,feminino
4,75,100,feminino


Para fazer a previsão do risco de cada um dos novos clientes, precisaremos transfomar o DataFrame para o mesmo formato ultilizado no treinamento do classificador.

In [13]:
#Prevendo o novo DataFrame
X_new_bin = onehot.transform(df_new.select_dtypes(include='object'))#incluindo a coluna que tenha o tipo primitivo('object') para fazer a lista binário.
X_new_num = minmax.transform(df_new.select_dtypes(exclude = 'object'))#Exlcuindo a coluna que tenha o 'object', fazendo a normalização.

#Juntando os 2 arrays
X_new = np.append(X_new_num, X_new_bin, axis=1)
X_new1 = pd.DataFrame(data = X_new, columns=['idade', 'conta-corrte', 'sexo'])

#Fazendo previsão
svc2.predict(X_new1)

display(X_new1)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


,idade,conta-corrte,sexo
0,0.033320,0.069964,1.0
1,0.122970,0.593814,0.0
2,0.571218,0.299148,1.0
3,0.302269,0.462851,0.0
4,1.019467,-0.044628,0.0


In [14]:
#Inserindo uma nova coluna 'previsão'
df_previsao = df_new.copy()
df_previsao['previsão'] = svc2.predict(X_new)
display(df_previsao)



,idade,conta_corrente,sexo,previsão
0,20,800,masculino,ruim
1,25,4000,feminino,bom
2,50,2200,masculino,ruim
3,35,3200,feminino,bom
4,75,100,feminino,ruim


Esse ultimo Dataframe foi inserido a coluna **Previsão**, fazendo um predição
se os novos clientes estão  está com a classificação de risco **bom** ou **ruim**.

In [15]:
#importando
from google.colab import files

# Exportando para um csv
df2.to_csv('Machine Learning - Parte_3.csv', index=False)
files.download('Machine Learning - Parte_3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>